In [1]:
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt
%pylab inline

from datetime import datetime,timedelta
import json
import re

from collections import OrderedDict
from math import pi
import pytz
import time
import psycopg2
from sqlalchemy import create_engine

from scipy import stats

from sklearn.decomposition import PCA
from sklearn.preprocessing import scale, normalize
import seaborn.apionly as sns

Populating the interactive namespace from numpy and matplotlib


In [2]:
import mmh3
import numpy as np
import pandas as pd
from scipy import sparse
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import explained_variance_score
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

/Users/linda/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/linda/anaconda2/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [3]:
data = pd.read_csv('balanced_data.csv')
data_im = pd.read_csv('imbalanced_data.csv')

/Users/linda/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
data_install = data[data['is_install'] == 1]
data_install_im = data_im[data_im['is_install'] == 1]

## Functions

In [90]:
def important_labels(colname,df,df_install):
    ins = pd.DataFrame(df_install.groupby(colname)['is_install'].count().rename('install_counts')).reset_index()
    total = pd.DataFrame(df.groupby(colname)['is_install'].count().rename('total_counts')).reset_index()
    counts = total.merge(ins, how = 'left', on = colname).fillna(0.0)
    counts['conversion'] = counts['install_counts']/counts['total_counts']
    #ins_im = pd.DataFrame(df_install_im.groupby(colname)['is_install'].count().rename('install_counts')).reset_index()
    #total_im = pd.DataFrame(df_im.groupby(colname)['is_install'].count().rename('total_counts')).reset_index()
    #counts_im = total_im.merge(ins_im, how = 'left', on = colname).fillna(0.0)
    #counts_im['conversion'] = counts_im['install_counts']/counts_im['total_counts']
    #sub_counts_im = counts_im[((counts_im.install_counts>1)&(counts_im.conversion>0.2))|((counts_im.install_counts>5)&(counts_im.conversion<0.05)) ]
    sub_counts = counts[((counts.install_counts>1)&(counts.conversion>0.5))|((counts.install_counts>5)&(counts.conversion<0.4))]
        
    return set(sub_counts[colname].values) #&set(sub_counts_im[colname].values)


In [6]:
def get_continent(x):
    if len(str(x).split('/')) > 1:
        return str(x).split('/')[0]
    else:
        return None
    
def get_city(x):
    if len(str(x).split('/')) > 1:
        return str(x).split('/')[1]
    else:
        return str(x).split('/')[0]

In [7]:
def logistic_model(X_train, y_train):
    """
    use crossvalidation (CV) to report the best parameter 'C'
    parameter C: Inverse of regularization strength; must be a positive float. 
    Check LogisticRegression() in sklearn for more information
    """
    print('Train Regression Model')
    model = GridSearchCV(
            estimator=LogisticRegression(),
            param_grid={'C': [0.01, 0.02, 0.05, 0.1, 0.2, 0.5]},
            scoring='log_loss',
            cv=5
    )
    model.fit(X_train, y_train)
    return model

In [57]:
def convert_to_unknown(df, columns):
    """
    convert NA to 'UNK'
    """
    if isinstance(df, pd.DataFrame):
        for col in columns:
            df[col].fillna("UNK", inplace=True)

    if isinstance(df, dict):
        for col in columns:
            df[col][pd.isnull(df[col])] = "UNK"
    return df
          
    
def fillna0(df, columns):
    """
    fill NA with 0
    """
    if isinstance(df, pd.DataFrame):
        for col in columns:
            df[col].fillna(0, inplace=True)

    if isinstance(df, dict):
        for col in columns:
            df[col][pd.isnull(df[col])] = 0    
    return df


def set_column_types(df, column_types_dict):
    if isinstance(column_types_dict, dict):
        for c, t in column_types_dict.items():
            df[c] = df[c].astype(t)
        return df 
    else:
        raise TypeError()

def preprocessing_data(df):
    """
    clean data
    """
    columns = {'is_install':int}
    df = set_column_types(df, columns)
    df = convert_to_unknown(df, ['advertiser_app_store_id',
                                 'country_code',
                                 'device_language',
                                 'device_platform',
                                 'time_zone'
                            ])
    df = fillna0(df, columns.keys())
    
    return df
  
class FeatureCreator():
    """Augment DataFrame-like input with new features."""

    def transform(self, X, inplace=False):
        # TODO probably it's a good idea to restrict what fields from the
        #      DataFrame are used to avoid copying the whole thing
        if isinstance(X, pd.DataFrame):
            X = {k: v.values for k, v in X.iteritems()}
        if not inplace:
            X = {k: np.copy(v) for k, v in X.iteritems()}
        return X

    def fit(self, X, y=None):
        return self

In [9]:
# Hashing function
def _murmur_32s(key, seed):

    if isinstance(key, unicode):
        bkey = key.encode('utf-8')
    elif isinstance(key, bytes):
        bkey = key
    else:
        print key
        print type(key)
        raise ValueError("the key must be either unicode or str")
    return mmh3.hash(bkey, seed)

  
# Hash features of DataFrame X using the hashing function
def _transform(X, n_bits, categorical_features,
              continuous_features, interaction_features,
              store_fmap=False):
    n_samples = X.shape[0] \
        if isinstance(X, pd.DataFrame) \
        else len(X.values()[0])
    hash_mask = 2 ** n_bits - 1
    n_features = \
        len(categorical_features) + \
        len(continuous_features) + \
        len(interaction_features)
    n_hashed_features = n_samples * n_features
    # assert n_hashed_features > 0
    rows = np.empty(n_hashed_features, dtype=np.int32)
    cols = np.empty(n_hashed_features, dtype=np.int32)
    vals = np.zeros(n_hashed_features)
    hashed_feature_idx = 0
    f_map = {}

    for f in categorical_features:
        Xf = X[f]
        hash_seed = _murmur_32s(f, 0)
        for sample_idx in range(n_samples):
            hash_value = _murmur_32s(Xf[sample_idx], hash_seed)
            hash_sign = (hash_value >= 0) * 2 - 1

            if store_fmap:
                f_combined = ((f,), Xf[sample_idx])
                if f_combined not in f_map:
                    f_map[f_combined] = hash_value & hash_mask

            rows[hashed_feature_idx] = sample_idx
            cols[hashed_feature_idx] = hash_value & hash_mask
            vals[hashed_feature_idx] += hash_sign
            hashed_feature_idx += 1

    for f in continuous_features:
        Xf = X[f]
        hash_value = _murmur_32s(f, 0)
        hash_sign = (hash_value >= 0) * 2 - 1
        if store_fmap:
            f_combined = ((f,),)
            f_map[f_combined] = hash_value & hash_mask
        for sample_idx in range(n_samples):
            rows[hashed_feature_idx] = sample_idx
            cols[hashed_feature_idx] = hash_value & hash_mask
            vals[hashed_feature_idx] += hash_sign * Xf[sample_idx]
            hashed_feature_idx += 1

    for feature_names in interaction_features:
        hash_seed = 0
        for f in feature_names:
            hash_seed = _murmur_32s(f, hash_seed)

        for sample_idx in range(n_samples):
            hash_value = hash_seed
            interaction_value = 1

            value_cache = ()
            for f in feature_names:
                if f in continuous_features:
                    interaction_value *= X[f][sample_idx]
                    value_cache += (f,)
                else:
                    value_cache += (X[f][sample_idx],)
                    hash_value = _murmur_32s(
                        X[f][sample_idx], hash_value
                    )

            if store_fmap:
                f_combined = (feature_names, value_cache)
                if f_combined not in f_map:
                    f_map[f_combined] = hash_value & hash_mask

            hash_sign = (hash_value >= 0) * 2 - 1
            rows[hashed_feature_idx] = sample_idx
            cols[hashed_feature_idx] = hash_value & hash_mask
            vals[hashed_feature_idx] += hash_sign * interaction_value
            hashed_feature_idx += 1

    n_dim_hashed_features = hash_mask + 1

    # reverse k and v, if v is duplicated, append k to v
    f_map_rev = {}
    for k, v in f_map.items():
        if v not in f_map_rev:
            f_map_rev[v] = [k]
        else:
            f_map_rev[v] = f_map_rev[v] + [k]

    return sparse.coo_matrix(
        (vals, (rows, cols)),
        (n_samples, n_dim_hashed_features)
    ).tocsr(), f_map_rev

# Wrapper class for hashing function
class FeatureHasher(BaseEstimator, TransformerMixin):
    def __init__(self,
                 n_bits=22,
                 categorical_features=None,
                 continuous_features=None,
                 interaction_features=None,
                 store_fmap=False):
        if n_bits < 1 or n_bits > 31:
            raise ValueError("number of bits must be in interval [1, 31]")

        self.n_bits_ = n_bits
        self.categorical_features_ = set(categorical_features or [])
        self.continuous_features_ = set(continuous_features or [])
        self.interaction_features_ = set(interaction_features or [])
        self.store_fmap = store_fmap

        n_features = len(self.categorical_features_) + \
                     len(self.continuous_features_) + \
                     len(self.interaction_features_)
        if n_features == 0:
            raise ValueError("at least one features needs to be specified")

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return _transform(
            X, self.n_bits_, self.categorical_features_,
            self.continuous_features_, self.interaction_features_,
            self.store_fmap
        )

## Clean data

In [10]:
cols = ['time_of_last_delivery_this_campaign', 'time_of_last_delivery_this_creative', 'time_of_last_delivery_any_installed_app',
       'time_of_last_delivery_any_installed_app','time_of_last_vungle_delivery', 'time_of_this_impression','time_of_this_request']
for col in cols:
#     print col
    data[col] =  pd.to_datetime(data[col]).dt.hour
    data_im[col] =  pd.to_datetime(data_im[col]).dt.hour

In [11]:
data['device_language'] = data['device_language'].apply(lambda x: str(x).lower().split('-')[0])
data_install['device_language'] = data_install['device_language'].apply(lambda x: str(x).lower().split('-')[0])
data_im['device_language'] = data_im['device_language'].apply(lambda x: str(x).lower().split('-')[0])
data_install_im['device_language'] = data_install_im['device_language'].apply(lambda x: str(x).lower().split('-')[0])

/Users/linda/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/linda/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [12]:
data['time_zone_continent'] = data['time_zone'].apply(lambda x: get_continent(x))
data_install['time_zone_continent'] = data_install['time_zone'].apply(lambda x: get_continent(x))
data_im['time_zone_continent'] = data_im['time_zone'].apply(lambda x: get_continent(x))
data_install_im['time_zone_continent'] = data_install_im['time_zone'].apply(lambda x: get_continent(x))

/Users/linda/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/linda/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


## Model Building

In [13]:
#might have device_language/country_code multicolinearity 
X_col = ['advertiser_app_store_id','country_code','n_campaign_views','device_language','device_platform','time_zone']
y = ['is_install']

In [14]:
data[X_col+y].copy().head()

,advertiser_app_store_id,country_code,n_campaign_views,device_language,device_platform,time_zone,is_install
0,1009442510,CN,4,zh,iOS,Asia/Shanghai,0
1,5743f032a5a36ff4300000a5,ID,0,id,android,Asia/Makassar,0
2,727296976,CA,0,en,iOS,America/Toronto,0
3,57a28ffbb019f8257c00021d,LA,1,th,iOS,Asia/Bangkok,0
4,com.plarium.vikings,CZ,0,cs,android,Europe/Prague,0


In [15]:
X_clf = preprocessing_data(data[X_col+y].copy())

### Classification Model Features - specify categorical, continuous, and interaction terms.

In [18]:
##you can't put integer column into this
## if you add something, please change the preprocessing function as well.
cat_features_pc = [
    'advertiser_app_store_id',
    'country_code',
    'device_language',
    'device_platform',
    'time_zone'
    ]

cont_features_pc = []

cat_interactions_pc = [
     ('advertiser_app_store_id',  'country_code'),
     ('country_code', 'device_language')
]

In [19]:
train, test = train_test_split(X_clf, test_size=0.2, random_state = 27)  # splitting Train into Train/Test sets
X_train, y_train, X_test, y_test = train, train.pop('is_install'), test, test.pop('is_install')

# Feature Hashing
print('Feature Creating/Hashing Train')
feature_creator = FeatureCreator()
design_matrix_transformer = FeatureHasher(
    18, cat_features_pc, None, cat_interactions_pc, store_fmap=True) # You can experiment with hasher bits (we used 18 here)

X_test = feature_creator.transform(X_test, inplace=True)
X_test, f_map = design_matrix_transformer.fit_transform(X_test)

X_train = feature_creator.transform(X_train, inplace=True)
X_train, f_map = design_matrix_transformer.fit_transform(X_train)

Feature Creating/Hashing Train


In [23]:
X_train.shape

(560000, 262144)

In [20]:
logistic_baseline = logistic_model(X_train, y_train)

# Calculate prediction/probability of train and test
X_train_predictions = logistic_baseline.predict(X_train)
X_train_predprob = logistic_baseline.predict_proba(X_train)[:, 1]

X_test_predictions = logistic_baseline.predict(X_test)
X_test_predprob = logistic_baseline.predict_proba(X_test)[:, 1]

# Calculate metrics of train, validation and test set.
lr_ll_val = -logistic_baseline.best_score_

lr_ll_train = log_loss(y_train, X_train_predprob)
lr_auc_train = roc_auc_score(y_train, X_train_predprob)

lr_ll_test = log_loss(y_test, X_test_predprob)    
lr_auc_test = roc_auc_score(y_test, X_test_predprob)

# Print out the results
print "Best parameter: ", logistic_baseline.best_params_

print "Log Loss (Validation): %f" % lr_ll_val

print "Log Loss (Train): %f" % lr_ll_train
print "AUC (Train): %f" % lr_auc_train

print 'Log Loss (Test): %f' % lr_ll_test
print 'AUC (Test): %f' % lr_auc_test

Train Regression Model


/Users/linda/anaconda2/lib/python2.7/site-packages/sklearn/metrics/scorer.py:127: DeprecationWarning: Scoring method log_loss was renamed to neg_log_loss in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/Users/linda/anaconda2/lib/python2.7/site-packages/sklearn/metrics/scorer.py:127: DeprecationWarning: Scoring method log_loss was renamed to neg_log_loss in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/Users/linda/anaconda2/lib/python2.7/site-packages/sklearn/metrics/scorer.py:127: DeprecationWarning: Scoring method log_loss was renamed to neg_log_loss in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/Users/linda/anaconda2/lib/python2.7/site-packages/sklearn/metrics/scorer.py:127: DeprecationWarning: Scoring method log_loss was renamed to neg_log_loss in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/Users/linda/anaconda2/lib/python2.7/site-packages/sklearn/metrics/scorer.py:127

Best parameter:  {'C': 0.2}
Log Loss (Validation): 0.596766
Log Loss (Train): 0.590061
AUC (Train): 0.751594
Log Loss (Test): 0.594483
AUC (Test): 0.746766


### Only create labels for one that is either over performed/ under performed

In [91]:
#might have device_language/country_code multicolinearity 
X_col = ['advertiser_app_store_id','country_code','device_language','device_platform','time_zone']

labels=[]
for i in X_col:
    labels.append(important_labels(i,data,data_install))

In [92]:
X_clf = preprocessing_data(data[X_col+y].copy())
train, test = train_test_split(X_clf, test_size=0.2, random_state = 27)  # splitting Train into Train/Test sets
X_train, y_train, X_test, y_test = train, train.pop('is_install'), test, test.pop('is_install')

In [93]:
for i in range(len(labels)):
    pattern = '|'.join(list(labels[0]))
    X_train.loc[~X_train[X_col[i]].str.contains(pattern),X_col[i]]='Other'+str(i)
    X_test.loc[~X_test[X_col[i]].str.contains(pattern),X_col[i]]='Other'+str(i)
    

/Users/linda/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/Users/linda/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [94]:
# Feature Hashing
print('Feature Creating/Hashing Train')
feature_creator = FeatureCreator()
design_matrix_transformer = FeatureHasher(
    18, cat_features_pc, None, cat_interactions_pc, store_fmap=True) # You can experiment with hasher bits (we used 18 here)

X_test = feature_creator.transform(X_test, inplace=True)
X_test, f_map = design_matrix_transformer.fit_transform(X_test)

X_train = feature_creator.transform(X_train, inplace=True)
X_train, f_map = design_matrix_transformer.fit_transform(X_train)

Feature Creating/Hashing Train


In [95]:
X_train.shape

(560000, 262144)

In [96]:
logistic_baseline = logistic_model(X_train, y_train)

# Calculate prediction/probability of train and test
X_train_predictions = logistic_baseline.predict(X_train)
X_train_predprob = logistic_baseline.predict_proba(X_train)[:, 1]

X_test_predictions = logistic_baseline.predict(X_test)
X_test_predprob = logistic_baseline.predict_proba(X_test)[:, 1]

# Calculate metrics of train, validation and test set.
lr_ll_val = -logistic_baseline.best_score_

lr_ll_train = log_loss(y_train, X_train_predprob)
lr_auc_train = roc_auc_score(y_train, X_train_predprob)

lr_ll_test = log_loss(y_test, X_test_predprob)    
lr_auc_test = roc_auc_score(y_test, X_test_predprob)

# Print out the results
print "Best parameter: ", logistic_baseline.best_params_

print "Log Loss (Validation): %f" % lr_ll_val

print "Log Loss (Train): %f" % lr_ll_train
print "AUC (Train): %f" % lr_auc_train

print 'Log Loss (Test): %f' % lr_ll_test
print 'AUC (Test): %f' % lr_auc_test

Train Regression Model


/Users/linda/anaconda2/lib/python2.7/site-packages/sklearn/metrics/scorer.py:127: DeprecationWarning: Scoring method log_loss was renamed to neg_log_loss in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/Users/linda/anaconda2/lib/python2.7/site-packages/sklearn/metrics/scorer.py:127: DeprecationWarning: Scoring method log_loss was renamed to neg_log_loss in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/Users/linda/anaconda2/lib/python2.7/site-packages/sklearn/metrics/scorer.py:127: DeprecationWarning: Scoring method log_loss was renamed to neg_log_loss in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/Users/linda/anaconda2/lib/python2.7/site-packages/sklearn/metrics/scorer.py:127: DeprecationWarning: Scoring method log_loss was renamed to neg_log_loss in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/Users/linda/anaconda2/lib/python2.7/site-packages/sklearn/metrics/scorer.py:127

Best parameter:  {'C': 0.5}
Log Loss (Validation): 0.609129
Log Loss (Train): 0.608264
AUC (Train): 0.728830
Log Loss (Test): 0.607242
AUC (Test): 0.730329


#### compare to when we have all the features without feature reduction

Best parameter:  {'C': 0.2}
Log Loss (Validation): 0.596766
Log Loss (Train): 0.590061
AUC (Train): 0.751594
Log Loss (Test): 0.594483
AUC (Test): 0.746766